In [1]:
from module.prepare import *
from itertools import product
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
# import redis
# import xgboost


def scoreFunction(result):
    scores = {}

#     for result in cv_results:
    acc = metrics.accuracy_score(result[0],result[1])
    auc = metrics.roc_auc_score(result[0],result[1])
    scores['acc'] = acc
    scores['auc'] = auc
    
    return scores



D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


#### LGB

In [2]:

'''
    @new_params whether to append new param
    @default whether to use default param
    @replace whether to replace old param
    @tuning whethe to tuning with GridSearchCV
    
    @return [Ypred,Ytest,score_train,clf]
'''
def LGBTuning(Xtrain,Xtest,Ytrain,Ytest,new_params=None,default=False,replace=False,tuning=True):
    
    clf = lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=12,
#                              class_weight
                            )
    
    gridParams = {
        # step 1
#     'learning_rate': [0.01,0.05,0.1],
#     'boosting_type':['gbdt','goss'],
#     'n_estimators': [50,200,500],
#     'num_iterations':[200,400,1000],
        # step 1 fixed
    'learning_rate': [0.1], ### 0.1
    'boosting_type':['gbdt'], ### goss>gbdt
    'n_estimators': [500],
    'num_iterations':[2000], ### 2000
#     'max_depth':[6], ### <6
        # step 2
#     'num_leaves': [200], ### <400<675
#     'min_data_in_leaf':[18,20,22], ### 20 default
#     'max_bin':[127,255,511],
        # step 2 fixed
#     'num_leaves': [800],
    'max_bin':[256],
        # step 3
#     'max_depth':[7,8,9,10], ### missed
    'colsample_bytree' : [0.9], ### 0.75
    'subsample_freq':[1], ### 1
        
#     'subsample' : [0.7], ### 1
#     'min_data_in_leaf':[26],
#     'early_stopping_round':[2],
#     'reg_alpha' : [1e-3], ### 0
#     'reg_lambda' : [0,0.1,0.5], ### 0
    }
    
    if replace is True:
        gridParams = {}
    if new_params is not None:
        gridParams.update(new_params)
    

    if tuning:
        grid = GridSearchCV(clf, gridParams,
                        scoring='accuracy',
                        verbose=3,
                        cv=5,
                        n_jobs=1)
        print('default params\n',clf.get_params())
        grid.fit(Xtrain,Ytrain)
        return grid
    else:
        if not default:
            arg_str = ''
            for k,v in gridParams.items():
                if type(v[0])==str:
                    arg_str += k+'='+"'"+v[0]+"',"
                else:
                    arg_str += k+'='+str(v[0])+","
            eval(
                'clf.'+clf.set_params.__name__+"("
                    +arg_str.rstrip(',')+
                    ")"
                )
#         clf.class_weight = {1:sum(Ytrain==1),0:sum(Ytrain==0)}
        print('params\n',clf.get_params())
        clf.fit(Xtrain,Ytrain)
        Ypred = clf.predict(Xtest)
        score_train = clf.score(Xtrain,Ytrain)
        print('train score %f'%score_train)
        return [Ypred,Ytest,score_train,clf]

In [ ]:
def LGBFit(X_train,X_test,Y_train,Y_test):
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_test = lgb.Dataset(X_test, Y_test, reference=lgb_train)

    params = {    
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'roc-auc',
#                 'nthread':6,
                'learning_rate':0.08,
                'num_leaves':300, 
                'max_depth': -1,   
                'subsample': 0.9, 
                'colsample_bytree': 0.9, 
#                 'feature_fraction': 1,
#                 'bagging_freq': 8,
# #                 'num_iterations':300,
#                 'min_data_in_leaf':2,
#                 'bagging_fraction': 0.8,
#                 'num_boost_round':3000,
            }

    cv_results = lgb.cv(params, lgb_train, nfold=5  
                        ,stratified=False, shuffle=True
                        ,seed=seed,
                        metrics=['auc','binary_logloss','mae']
                        ,verbose_eval=1)
    print('best n_estimators:', len(cv_results['auc-mean']))
    for k,v in cv_results.items():
        print('best cv score:', k, pd.Series(cv_results[k]).max())
    return [lgb,cv_results]

#### main

In [3]:
'''
'data_NPInter10412',
'reRPI2825',
'RPI488',
'RPI2241',
'RPITER_RPI1807'
'''

hyper_params = GetConfigure()
num_hyper_params = len(hyper_params)
lassocv_param = {'threshold':0.05}

cv = 5
generalize_ratio = 1.0/cv
test_ratio = 1.0/cv

mi_use = False
outside_grid = True
tuning_mode = False

if tuning_mode:
    cv = 1

cv_results = []
# scores = []
'''
    topK =3000
'''


search_param = [{'learning_rate':[0.05,0.1,0.01,0.02],
                'colsample_bytree':[0.7,0.8,0.9,1],
                 'max_depth':[5,6,7,8,9],
                 'num_iterations':[500,1000,2000],
#                  'min_data_in_leaf':[20,25,30]
                }]
search_grid = list(ParameterGrid(search_param))

conf_param = {}


cfile = GetConfigureObject()
commons = dict(cfile.items('common'))

# db = redis.StrictRedis(host=commons[str.lower('REDIS_HOST')], port=6379, db=0, 
#                       password=commons[str.lower('REDIS_PWD')], decode_responses=True
#                       )



#### tuning stage 1

In [ ]:
def TuningParametersStage1():
    res = []
    for DATAID in [0,1,2,3,4]:
        INFO('data id %d'%DATAID)
        for RNA_K in range(3,7):
            for PROTEIN_K in range(3,7):
                for TOP_RATIO in np.linspace(0.93,0.99,5):
                    [data,T] = ReadData(DATAID,PROTEIN_K,RNA_K)
                    [X,Y] = ToMatrix(data,'dense')
                    [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
                    [X_train,X_test,Y_train,Y_test] = \
                                RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=0.96)
                    r = LGBTuning(X_train,X_test,Y_train,Y_test,tuning=False,default=True)
                    r = {'test_score':scoreFunction([r[0],r[1]]),
                        'train_score':r[2],
                        'RNA_K':RNA_K,
                        'PROTEIN_K':PROTEIN_K,
                        'TOP_RATIO':TOP_RATIO}
                    res.append(r)
                    WriteDictResult(DATAID,r,'3-14-result-of-k')
    return


In [ ]:
def getTuningStage1Result():
    fname = './3-14-result-of-k'
    with open(fname+'.txt','r+') as f:
        text = f.read()
        content = text.split('\n')


    df = pd.DataFrame()
    count = 0
    for line in content:
        count += 1
        if len(line):
            if line[0] is '@':
                if count>1:
                    df = df.append([
                        [
                           r['DATAID'],
                           r['RNA_K'],
                           r['PROTEIN_K'],
                           r['TOP_RATIO'],
                           eval(r['test_score'])['acc'],
                           eval(r['test_score'])['auc'],
                           r['train_score']
                        ]
                                   ])
                r = {}
                r['DATAID'] = line[1]
            else:
                line = line.split('=\t')
                if line[0] is 'test_score':
                    r['test_score'] = dict(eval(line[1]))
                else:
                    r[line[0]] = line[1]

    df.columns=['DATAID','RNA_K','PROTEIN_K',
               'TOP_RATIO',
               'acc','auc','train_score']
    return df
df = getTuningStage1Result()

#### tuning stage 2

In [4]:
fname_optimal_stage1 = './3-14-data-param-1.csv'
fname_result = './3-16-param-tune-2.csv'

tune_grid = [
    [{
        'learning_rate':[0.05,0.1,0.01,0.02],
#         'colsample_bytree':[0.7,0.8,0.9,1],
        'max_depth':[-1,5,6,7,8,9],
        'num_iterations':[500,1000,2000],
    }],
    [{
        'learning_rate':[0.05],
        'colsample_bytree':[0.7,0.8,0.9,1],
#         'max_depth':[-1,5,6,7,8,9],
        'min_data_in_leaf':[18,20,22,30],
        'num_iterations':[1000],
        'num_leaves':[100,200,300,500],
        'lambda_l1':[0,1e-3,1e-1],
    }],
    [{
        'learning_rate':[0.05],
        'colsample_bytree':[0.7,0.8,0.9,1],
#         'max_depth':[-1,5,6,7,8,9],
        'min_data_in_leaf':[18,20,22,30],
        'num_iterations':[1000],
        'num_leaves':[100,200,300,500],
        'lambda_l1':[0,1e-3,1e-1],
    }],
    [{
        'learning_rate':[0.1],
        'colsample_bytree':[0.7,0.8,0.9,1],
        'max_depth':[-1],
        'min_data_in_leaf':[18,20,22,30],
        'num_iterations':[2000],
        'num_leaves':[100,200,300,500],
        'lambda_l1':[0,1e-3,1e-1],
    }],
    [{
        'learning_rate':[0.01],
        'colsample_bytree':[0.7,0.8,0.9,1],
        'max_depth':[5,6,7],
        'min_data_in_leaf':[18,20,22,30],
        'num_iterations':[1000],
        'num_leaves':[100,200,300,500],
        'lambda_l1':[0,1e-3,1e-1],
    }]
]

tune_grid = list(map(lambda x:list(ParameterGrid(x)),tune_grid))

tuning_cv2 = 3
tuning_generalize_ratio2 = 1.0/tuning_cv2
df_optimal = pd.read_csv(fname_optimal_stage1)

df_result2 = pd.DataFrame([],columns=['dataid',
                                      'cv',
                                      'training_score',
                                      'tune_param',
                                      'acc',
                                      'auc'
                                     ])

for _dataid in [1,2,3,4]:
    INFO('dataid %d'%_dataid)
    ### get conf of dataid
    df_current = df_optimal.loc[df_optimal['DATAID']==_dataid]
    rna_k = df_current['RNA_K'].to_numpy()[0]
    protein_k = df_current['PROTEIN_K'].to_numpy()[0]
    top_ratio = df_current['TOP_RATIO'].to_numpy()[0]
    ### read data
    [data,T] = ReadData(_dataid,protein_k,rna_k)
    [X,Y] = ToMatrix(data,'dense')
    ### split dataset
    [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,tuning_generalize_ratio2)
    ### dimensionality reduction
    [X_train,X_test,Y_train,Y_test] = \
                RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=top_ratio)
    for _cv in range(1):
        INFO('tuning cv %d'%_cv)
        for sp in tune_grid[_dataid]:
            sp = dict(map(lambda x:(x,[sp[x]]),sp))
            tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,sp,tuning=False)
            tune_score = scoreFunction(tune_results)
            r = pd.Series({'dataid':_dataid,
                 'cv':_cv,
                 'training_score':tune_results[2],
                 'tune_param':str(sp),
                 'acc':tune_score['acc'],
                 'auc':tune_score['auc'],
                })
            df_result2 = df_result2.append(r,ignore_index=True)
            
df_result2.to_csv(fname_result)
            

INFO::dataid 1
read data 3 4
=============RETRIEVE TRIAN DATA=================
# DEBUG: # DEBUG: **************new dl 1***************
# DEBUG: READ SEQ FROM FILE
# DEBUG: READ CLUSTER FROM FILE
regex error 
regex error 
# DEBUG: READ PAIR FROM FILE
# DEBUG: GENERATE NEGATIVE PAIR
# DEBUG: negative pair number 2825
INFO::count of negative pairs2825
# DEBUG: PAIR UNION
# DEBUG: EXTRACT FEATURES--PROTEIN
# DEBUG: EXTRACT FEATURES--RNA
# DEBUG: K-MER CALCULATION
# DEBUG: FEATURE UNION
# DEBUG: GARBAGE COLLECTION
MATRIX TRANSFORMATION
data shape 5650 3137
rf raw data fit score 1.000000
INFO::dimension remained 3024 0.930000
dimension remained 3024
INFO::tuning cv 0
params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.7, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 12, 'num_leaves': 100, 'objective': 'cross_entropy', 'random_state': 42, 'r

D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 1.000000
params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.7, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 12, 'num_leaves': 200, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'verbose': 1, 'num_iterations': 1000, 'max_bin': 256, 'lambda_l1': 0, 'min_data_in_leaf': 18}


D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 1.000000
params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.7, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 12, 'num_leaves': 300, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'verbose': 1, 'num_iterations': 1000, 'max_bin': 256, 'lambda_l1': 0, 'min_data_in_leaf': 18}


...

train score 0.989277
params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1, 'importance_type': 'split', 'learning_rate': 0.01, 'max_depth': 7, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 12, 'num_leaves': 300, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'verbose': 1, 'num_iterations': 1000, 'max_bin': 256, 'lambda_l1': 0.1, 'min_data_in_leaf': 30}


D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 0.989277
params
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1, 'importance_type': 'split', 'learning_rate': 0.01, 'max_depth': 7, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': 12, 'num_leaves': 500, 'objective': 'cross_entropy', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'verbose': 1, 'num_iterations': 1000, 'max_bin': 256, 'lambda_l1': 0.1, 'min_data_in_leaf': 30}


D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


train score 0.989277


In [ ]:
key = 'auc'
df_temp = df.groupby(by=['DATAID']) \
                .aggregate({key:max}).reset_index()

df_result = df.join(df_temp.set_index(['DATAID',key]),on=['DATAID',key],how='inner')
df_result.to_csv('./3-14-data-param-1.csv')

In [ ]:
df.loc[(df['DATAID']=='0')
      & (df['RNA_K']=='3')
      & ( (df['PROTEIN_K']=='3') | (df['PROTEIN_K']=='4') )]

In [ ]:




for batch in range(cv):
    INFO('cross validation batch %d'%batch)
    if mi_use==True:
        arr = ToMatrix(data,'sparse')
        [X_train,X_test,Y_train,Y_test] = MutualInformationFeatureSelection2(arr,data,generalize_ratio)
        [X_train,X_test,Y_train,Y_test] = \
            RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test)
    else:
        [X,Y] = ToMatrix(data,'dense')
        [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
#         X_train,X_test,Y_train,Y_test = IsomapDimensionalityReduction(X_train,X_test,Y_train,Y_test)
#         [X_train,X_test,Y_train,Y_test] = LassoCVFeatureSelection(X_train,X_test,Y_train,Y_test,lassocv_param)
        [X_train,X_test,Y_train,Y_test] = \
            RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test)
#         [X_train,X_test,Y_train,Y_test] = \
#             PCADimensionalityReduction(X_train,X_test,Y_train,Y_test)
    if tuning_mode:
        [Xtrain,Ytrain] = merge_train_test(X_train,X_test,Y_train,Y_test)
        grid = LGBTuning(Xtrain,[],Ytrain,[],True)
        cv_results.append(grid)
        break
    else:
        if outside_grid is True:
            for sp in search_grid:
                sp = dict(map(lambda x:(x,[sp[x]]),sp))
                tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,sp,tuning=False)
                r = {'batch':batch,'res':tune_results,'sp':sp,'score':scoreFunction(tune_results)}
                cv_results.append(r)
            break
        else:
            tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,tuning=False,default=False)
            r = {'batch':batch,'res':tune_results,'score':scoreFunction(tune_results)}
            cv_results.append(r)
        


In [ ]:
### compute scores
scores = [scoreFunction(r['res'])for r in cv_results]
auc = np.mean([x['auc'] for x in scores])
acc = np.mean([x['acc'] for x in scores])
scores,auc,acc

In [ ]:
cv_results

In [ ]:

### save model
if outside_grid is True:
    search_param = set( list(map(lambda x:x['sp'],cv_results)) )
    scores = [{sp:[] for sp in search_param}]
    for sp in search_param:
        scores[sp] = np.mean(list( map(lambda x:x['score'],filter(lambda x:x['sp']==sp,cv_results)) ))
    best_sp = sorted(scores,key=lambda x:scores[x][0],reverse=True)[0]['sp']
    


#### test

In [ ]:
DATAID = 1
PROTEIN_K = 3
RNA_K = 4
topRatio = 0.93
[data,T] = ReadData(DATAID,PROTEIN_K,RNA_K)
[X,Y] = ToMatrix(data,'dense')
[X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
[X_train,X_test,Y_train,Y_test] = \
    RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=0.96)
r1 = LGBTuning(X_train,X_test,Y_train,Y_test,tuning=False,default=True)

estimators = [
    ('rf', RandomForestClassifier(n_estimators=1000, random_state=seed)),
    ('lgb', lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=12,
                            )),
    ('xgb',xgboost.XGBClassifier())
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)


In [ ]:
clf.fit(X_train,Y_train).score(X_test,Y_test)

In [ ]:
scoreFunction([r1[0],r1[1]])

#### tuning

In [ ]:
scores = [score for grid in cv_results for score in grid.cv_results_['mean_test_score']]
sns.distplot(scores,rug=True,bins=20)
plt.show()

# param_rank = np.array([grid.cv_results_['mean_test_score'],grid.cv_results_['params']]).T
# a = sorted(param_rank,key=lambda x:x[0],reverse=True)
# a = np.array(list(a))

# a
max( cv_results,key=lambda x:np.mean(x.cv_results_['mean_test_score']) ).best_params_

In [ ]:
feature_importance = list( sorted(grid.best_estimator_.feature_importances_,reverse=True) )
sum(feature_importance[:2000])/sum(feature_importance)

plt.figure()
sns.distplot(grid.best_estimator_.feature_importances_,bins=100)
plt.xlabel('feature importance')
plt.ylabel('ratio')
plt.show()